In [1]:
import os
DEVELOPMENT_MODE = False
# Detect if we're running in Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
except:
    IN_COLAB = False

# Install if in Colab
if IN_COLAB:
    %pip install transformer_lens
    %pip install circuitsvis
    # Install a faster Node version
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs  # noqa

# Hot reload in development mode & not running on the CD
if not IN_COLAB:
    from IPython import get_ipython
    ip = get_ipython()
    if not ip.extension_manager.loaded:
        ip.extension_manager.load('autoreload')
        %autoreload 2

IN_GITHUB = os.getenv("GITHUB_ACTIONS") == "true"
print("works")


works


In [2]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio
if IN_COLAB or not DEVELOPMENT_MODE:
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "notebook_connected"
print(f"Using renderer: {pio.renderers.default}")

Using renderer: colab


In [3]:
# Import stuff
import torch
import torch.nn as nn
import einops
from fancy_einsum import einsum
import tqdm.auto as tqdm
import plotly.express as px

from jaxtyping import Float
from functools import partial

/home/jennyni/miniconda3/envs/vlm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



In [4]:
# import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, FactoredMatrix

In [5]:
torch.set_grad_enabled(False)

In [6]:
device = utils.get_device()

In [ ]:
model = HookedTransformer.from_pretrained("gpt2", device=device)

Loaded pretrained model gpt2 into HookedTransformer


In [10]:
print(model)

HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0-11): 12 x TransformerBlock(
      (ln1): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_in): HookPoint()
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_out): HookPoint()
      (h

In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer

test = AutoModelForCausalLM.from_pretrained("gpt2")
print(test)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)


In [12]:
model_path = r"C:\Users\allan\ResearchStuff\checkpoint-1953"

from transformers import GPTJForCausalLM, AutoTokenizer
pretrained_model = GPTJForCausalLM.from_pretrained(model_path)

In [13]:
print(pretrained_model)

GPTJForCausalLM(
  (transformer): GPTJModel(
    (wte): Embedding(50257, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPTJBlock(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): Linear(in_features=768, out_features=768, bias=False)
          (v_proj): Linear(in_features=768, out_features=768, bias=False)
          (q_proj): Linear(in_features=768, out_features=768, bias=False)
          (out_proj): Linear(in_features=768, out_features=768, bias=False)
        )
        (mlp): GPTJMLP(
          (fc_in): Linear(in_features=768, out_features=3072, bias=True)
          (fc_out): Linear(in_features=3072, out_features=768, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((7

In [8]:
from transformer_lens.HookedTransformerConfig import HookedTransformerConfig
import math

config = {
    "d_model" : 2048, 
    "d_head" : 128, 
    "n_heads" : 16, 
    "d_mlp" : 8192,
    "n_layers" : 28, 
    "n_ctx" : 2048,
    "eps" : 1e-5, 
    "d_vocab" : 50257, 
    "act_fn" : "gelu", 
    "use_attn_scale" : math.sqrt(128),
    "use_local_attn" : False, 
    "scale_attn_by_inverse_layer_idx": False,
    "parallel_attn_mlp": True,
    "positional_embedding_type": "rotary",
    "rotary_dim": 64,
    "rotary_adjacent_pairs": True,
    "normalization_type": "LN",
    "model_name" : "custom",
    "original_architecture" : "gptj",
    }



In [ ]:
"""        model_name (str): the name of the model, used to load
            weights from HuggingFace or initialized to "custom" if not passed
        original_architecture (str, *optional*): the family of the model, used
        to help load
            weights from HuggingFace or initialized to "custom" if not passed
        from_checkpoint (bool): Whether the model weights were
            loaded from a checkpoint (only applies to pretrained models)
        checkpoint_index (int, *optional*): The index of the
            checkpoint loaded (only applies to pretrained models).
        checkpoint_label_type (str, *optional*): Whether
            checkpoints are labelled by the number of steps or number of tokens.
        checkpoint_value (int, *optional*): The value of the
            checkpoint label (whether of steps or tokens).
        tokenizer_name (str, *optional*): the full name of the model, passed into
            HuggingFace to access the tokenizer. Only used when passing in
            custom config, if loading from pretrained then this is not needed."""

In [9]:
model = HookedTransformer(config)

In [10]:
print(model)

HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (blocks): ModuleList(
    (0-27): 28 x TransformerBlock(
      (ln1): LayerNorm(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): LayerNorm(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
        (hook_rot_k): HookPoint()
        (hook_rot_q): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_in): HookPoint()
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_out): HookPoint()
    

In [19]:
CHECKPOINT_DIR = "/home/sjoshi/lmm/lmm_synthetic/lm_train/checkpoints/v3_spatial_grid_gptj/checkpoint-1953"

In [21]:
import json  

with open(f"{CHECKPOINT_DIR}/config.json", "r") as f:
    config = json.load(f)

# create the hooked transfomers

In [22]:
import os 

os.listdir(CHECKPOINT_DIR)

['config.json',
 'generation_config.json',
 'model.safetensors',
 'tokenizer_config.json',
 'special_tokens_map.json',
 'added_tokens.json',
 'vocab.json',
 'merges.txt',
 'tokenizer.json',
 'training_args.bin',
 'global_step1953',
 'zero_to_fp32.py',
 'latest',
 'scheduler.pt',
 'rng_state_1.pth',
 'rng_state_6.pth',
 'rng_state_2.pth',
 'rng_state_7.pth',
 'rng_state_3.pth',
 'rng_state_5.pth',
 'rng_state_0.pth',
 'rng_state_4.pth',
 'trainer_state.json']

In [25]:
import safetensors.torch 

state_dict = safetensors.torch.load_file(f"{CHECKPOINT_DIR}/model.safetensors")

In [26]:
model.load_state_dict(state_dict)

RuntimeError: Error(s) in loading state_dict for HookedTransformer:
	Missing key(s) in state_dict: "embed.W_E", "blocks.0.ln1.w", "blocks.0.ln1.b", "blocks.0.ln2.w", "blocks.0.ln2.b", "blocks.0.attn.W_Q", "blocks.0.attn.W_O", "blocks.0.attn.b_Q", "blocks.0.attn.b_O", "blocks.0.attn.W_K", "blocks.0.attn.W_V", "blocks.0.attn.b_K", "blocks.0.attn.b_V", "blocks.0.attn.mask", "blocks.0.attn.IGNORE", "blocks.0.attn.rotary_sin", "blocks.0.attn.rotary_cos", "blocks.0.mlp.W_in", "blocks.0.mlp.b_in", "blocks.0.mlp.W_out", "blocks.0.mlp.b_out", "blocks.1.ln1.w", "blocks.1.ln1.b", "blocks.1.ln2.w", "blocks.1.ln2.b", "blocks.1.attn.W_Q", "blocks.1.attn.W_O", "blocks.1.attn.b_Q", "blocks.1.attn.b_O", "blocks.1.attn.W_K", "blocks.1.attn.W_V", "blocks.1.attn.b_K", "blocks.1.attn.b_V", "blocks.1.attn.mask", "blocks.1.attn.IGNORE", "blocks.1.attn.rotary_sin", "blocks.1.attn.rotary_cos", "blocks.1.mlp.W_in", "blocks.1.mlp.b_in", "blocks.1.mlp.W_out", "blocks.1.mlp.b_out", "blocks.2.ln1.w", "blocks.2.ln1.

In [27]:
print(state_dict.keys())

dict_keys(['lm_head.bias', 'lm_head.weight', 'transformer.h.0.attn.k_proj.weight', 'transformer.h.0.attn.out_proj.weight', 'transformer.h.0.attn.q_proj.weight', 'transformer.h.0.attn.v_proj.weight', 'transformer.h.0.ln_1.bias', 'transformer.h.0.ln_1.weight', 'transformer.h.0.mlp.fc_in.bias', 'transformer.h.0.mlp.fc_in.weight', 'transformer.h.0.mlp.fc_out.bias', 'transformer.h.0.mlp.fc_out.weight', 'transformer.h.1.attn.k_proj.weight', 'transformer.h.1.attn.out_proj.weight', 'transformer.h.1.attn.q_proj.weight', 'transformer.h.1.attn.v_proj.weight', 'transformer.h.1.ln_1.bias', 'transformer.h.1.ln_1.weight', 'transformer.h.1.mlp.fc_in.bias', 'transformer.h.1.mlp.fc_in.weight', 'transformer.h.1.mlp.fc_out.bias', 'transformer.h.1.mlp.fc_out.weight', 'transformer.h.10.attn.k_proj.weight', 'transformer.h.10.attn.out_proj.weight', 'transformer.h.10.attn.q_proj.weight', 'transformer.h.10.attn.v_proj.weight', 'transformer.h.10.ln_1.bias', 'transformer.h.10.ln_1.weight', 'transformer.h.10.mlp.f